In [1]:
%reload_ext autoreload
%autoreload 1
import torch 
import sys
sys.path.append('..')
from torch import nn 
from torch.nn import functional as F
from torch import optim
from utils.loader import load
from utils.loader import PairSetMNIST
from torch.utils.data import Dataset, DataLoader

In [2]:
# load the dataset as a Dataset object
train_data = PairSetMNIST(train=True, swap_channel = True)
test_data  = PairSetMNIST(test=True)

In [10]:
class LeNet_aux_sequential(nn.Module):
    """
    Weight sharing + Auxiliary loss
    
    """
    def __init__(self):
        super(LeNet_aux_sequential, self).__init__()
        # convolutional weights for digit reocgnition shared for each image
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.dropout = nn.Dropout()
        
        # weights for binary classification 
        self.fc3 = nn.Linear(20, 60)
        self.fc4 = nn.Linear(60, 90)
        self.fc5 = nn.Linear(90, 2)
        
    def forward(self, input_):    
        
        # split the 2-channel input into two 14*14 images
        x = input_[:, 0, :, :].view(-1, 1, 14, 14)
        y = input_[:, 1, :, :].view(-1, 1, 14, 14)
        
        # forward pass for the first image 
        x = F.relu(F.max_pool2d(self.bn1(self.conv1(x)), kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.bn2(self.conv2(x)), kernel_size=2, stride=2))
        x = F.relu(self.dropout(self.fc1(x.view(-1, 256))))
        x = self.dropout(self.fc2(x))
        
        # forward pass for the second image 
        y = F.relu(F.max_pool2d(self.bn1(self.conv1(y)), kernel_size=2, stride=2))
        y = F.relu(F.max_pool2d(self.bn2(self.conv2(y)), kernel_size=2, stride=2))
        y = F.relu(self.dropout(self.fc1(y.view(-1, 256))))
        y = self.dropout(self.fc2(y))
        
        # concatenate layers  
        z = torch.cat([x, y], 1)
        
        z = F.relu(self.dropout(self.fc3(z)))
        z = F.relu(self.dropout(self.fc4(z)))
        z = self.dropout(self.fc5(z))
        
        return x, y, z

In [11]:
##### train function ######

def train_aux (model, train_data, mini_batch_size=100, optimizer = optim.SGD,
                criterion = nn.CrossEntropyLoss(), n_epochs=50, eta=1e-1, lambda_l2 = 0, alpha=1.0, beta=0.5):
    
    
    """
    Train network with auxiliary loss + weight sharing
    
    """
    # create data loader
    train_loader = DataLoader(train_data, batch_size=mini_batch_size, shuffle=True)
    
    model.train()
    optimizer = optimizer(model.parameters(), lr = eta)
    
    for e in range(n_epochs):
        epoch_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            input_, target_, classes_ = data
            class_1, class_2, out = model(input_)
            aux_loss1 = criterion(class_1, classes_[:,0])
            aux_loss2 = criterion(class_2, classes_[:,1])
            out_loss  = criterion(out, target_)
            net_loss = (alpha * (out_loss) + beta * (aux_loss1 + aux_loss2) )
            epoch_loss += net_loss
            
            if lambda_l2 != 0:
                for p in model.parameters():
                    epoch_loss += lambda_l2 * p.pow(2).sum() # add an l2 penalty term to the loss 
            
            optimizer.zero_grad()
            net_loss.backward()
            optimizer.step()
            
        print('Train Epoch: {}  | Loss {:.6f}'.format(
                e, epoch_loss.item()))
        
#########################################################################################################################
#########################################################################################################################

### test function  ###

def test_aux(model, test_data, mini_batch_size=100, criterion = nn.CrossEntropyLoss()):
    
    """
    Test function to calculate prediction accuracy of a cnn with auxiliary loss
    
    """
    
    # create test laoder
    test_loader = DataLoader(test_data, batch_size=mini_batch_size, shuffle=True)
    
    model.eval()
    test_loss = 0
    nb_errors=0
    
    with torch.no_grad():
        
        for i, data in enumerate(test_loader, 0):
            input_, target_, classes_ = data
            
            _, _, output = model(input_) 
            batch_loss = criterion(output, target_)
            test_loss += batch_loss
            
            _, predicted_classes = output.max(1)
            for k in range(mini_batch_size):
                if target_[k] != predicted_classes[k]:
                    nb_errors = nb_errors + 1
                                   
             
        print('\nTest set | Loss: {:.4f} | Accuracy: {:.0f}% | # misclassified : {}/{}\n'.format(
        test_loss.item(), 100 * (len(test_data)-nb_errors)/len(test_data), nb_errors, len(test_data)))
        

In [12]:
model = LeNet_aux_sequential()
train_aux(model, train_data)
test_aux(model,test_data)

Train Epoch: 0  | Loss 51.101402
Train Epoch: 1  | Loss 41.135033
Train Epoch: 2  | Loss 38.455769
Train Epoch: 3  | Loss 35.960327
Train Epoch: 4  | Loss 34.949295
Train Epoch: 5  | Loss 34.027515
Train Epoch: 6  | Loss 33.251362
Train Epoch: 7  | Loss 32.348801
Train Epoch: 8  | Loss 31.669588
Train Epoch: 9  | Loss 31.171227
Train Epoch: 10  | Loss 31.015421
Train Epoch: 11  | Loss 30.634390
Train Epoch: 12  | Loss 29.963057
Train Epoch: 13  | Loss 29.956802
Train Epoch: 14  | Loss 29.242424
Train Epoch: 15  | Loss 28.836374
Train Epoch: 16  | Loss 29.415770
Train Epoch: 17  | Loss 28.112406
Train Epoch: 18  | Loss 28.229288
Train Epoch: 19  | Loss 27.410818
Train Epoch: 20  | Loss 28.271755
Train Epoch: 21  | Loss 28.199747
Train Epoch: 22  | Loss 27.989101
Train Epoch: 23  | Loss 27.285320
Train Epoch: 24  | Loss 28.256323
Train Epoch: 25  | Loss 26.620724
Train Epoch: 26  | Loss 26.752785
Train Epoch: 27  | Loss 27.025396
Train Epoch: 28  | Loss 27.165651
Train Epoch: 29  | Loss 